In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
os.getcwd()

'd:\\Dropbox\\Self-Development\\Coding_Projects\\TextSummarizer'

In [62]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path
    input_feature: str
    target_feature: str

In [63]:
from src.constants import *
from src.utils.exception import CustomException
from src.utils.logger import logger
from src.utils.utils import load_yaml, create_directories
#from src.entity import DataTransformationConfig


class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = load_yaml(config_filepath)
        self.params = load_yaml(params_filepath)

        create_directories([self.config["artifacts_root"]])
        
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config["data_transformation"]

        create_directories([config["root_dir"]])

        data_transformation_config = DataTransformationConfig(
            root_dir=config["root_dir"],
            data_path=config["data_path"],
            tokenizer_name=config["tokenizer_name"],
            input_feature=config["input_feature"],
            target_feature=config["target_feature"]
        )

        return data_transformation_config

In [64]:
import os
import sys
from src.utils.exception import CustomException
from src.utils.logger import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)
    
    def convert_examples_to_features(self, batch):
        input_encodings = self.tokenizer(batch[self.config.input_feature], max_length=1024, truncation=True)
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(batch[self.config.target_feature], max_length=1024, truncation=True)
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'label': target_encodings['input_ids']
        }
        
    def convert(self):
        dataset = load_from_disk(self.config.data_path)
        dataset_transformed = dataset.map(self.convert_examples_to_features, batched=True)
        logger.info("Dataset transformation done.")
        dataset_transformed.save_to_disk(os.path.join(self.config.root_dir, 'transformed_dataset'))
        logger.info("Transformed dataset saved")

In [65]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise CustomException(e, sys)

[2023-08-06 02:08:58,760: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2023-08-06 02:08:58,761: INFO: utils: yaml file: params.yaml loaded successfully]
[2023-08-06 02:08:58,762: INFO: utils: created directory at: artifacts]
[2023-08-06 02:08:58,763: INFO: utils: created directory at: artifacts/data_transformation]


d:\Dropbox\Self-Development\Coding_Projects\TextSummarizer\venv\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jinou\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]d:\Dropbox\Self-Development\Coding_Projects\TextSummarize

[2023-08-06 02:09:03,914: INFO: 38029408: Dataset transformation done.]



Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 74364.19 examples/s]

[2023-08-06 02:09:03,989: INFO: 38029408: Transformed dataset saved]
